In [79]:
import os
import pandas as pd
import pm4py
from pm4py.algo.simulation.playout.petri_net import algorithm as playout_simulator
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.objects.conversion.log.variants import to_data_frame

In [8]:
path = "running-example.xes"

In [9]:
if not os.path.exists(path):
    raise FileNotFoundError(f"File not found: {path}")

# Read the XES log
log = pm4py.read_xes(path)

# Basic counts
n_events = len(log)


print(f"Number of events: {n_events}")

parsing log, completed traces :: 100%|██████████| 6/6 [00:00<00:00, 1933.01it/s]

Number of events: 42


In [10]:
net, im, fm = alpha_miner.apply(log)

In [71]:
new_log = playout_simulator.apply(net, im, fm, parameters={"noTraces": 500})

In [72]:
new_log

[{'attributes': {'concept:name': '0'}, 'events': [{'concept:name': 'register request', 'time:timestamp': datetime.datetime(1970, 4, 26, 18, 46, 40, tzinfo=datetime.timezone.utc)}, '..', {'concept:name': 'reject request', 'time:timestamp': datetime.datetime(1970, 4, 26, 18, 46, 48, tzinfo=datetime.timezone.utc)}]}, '....', {'attributes': {'concept:name': '499'}, 'events': [{'concept:name': 'register request', 'time:timestamp': datetime.datetime(1970, 4, 26, 19, 44, 19, tzinfo=datetime.timezone.utc)}, '..', {'concept:name': 'reject request', 'time:timestamp': datetime.datetime(1970, 4, 26, 19, 44, 27, tzinfo=datetime.timezone.utc)}]}]

In [73]:
new_log_df = to_data_frame.apply(new_log)

#pm4py.write_xes(new_log , "simulated_log.xes")

In [76]:
new_log_df

,concept:name,time:timestamp,case:concept:name
0,register request,1970-04-26 18:46:40+00:00,0
1,examine thoroughly,1970-04-26 18:46:41+00:00,0
2,check ticket,1970-04-26 18:46:42+00:00,0
3,decide,1970-04-26 18:46:43+00:00,0
4,reinitiate request,1970-04-26 18:46:44+00:00,0
...,...,...,...
3463,reinitiate request,1970-04-26 19:44:23+00:00,499
3464,examine thoroughly,1970-04-26 19:44:24+00:00,499
3465,check ticket,1970-04-26 19:44:25+00:00,499
3466,decide,1970-04-26 19:44:26+00:00,499


In [82]:
new_log_df.groupby('case:concept:name').agg(
    event_count = pd.NamedAgg(column='concept:name', aggfunc='count')
).mean()

event_count    6.936
dtype: float64

In [86]:
new_log_df[new_log_df['concept:name'] == "reject request"]

,concept:name,time:timestamp,case:concept:name
8,reject request,1970-04-26 18:46:48+00:00,0
13,reject request,1970-04-26 18:46:53+00:00,1
18,reject request,1970-04-26 18:46:58+00:00,2
23,reject request,1970-04-26 18:47:03+00:00,3
28,reject request,1970-04-26 18:47:08+00:00,4
...,...,...,...
3380,reject request,1970-04-26 19:43:00+00:00,488
3407,reject request,1970-04-26 19:43:27+00:00,491
3412,reject request,1970-04-26 19:43:32+00:00,492
3434,reject request,1970-04-26 19:43:54+00:00,494
